RE -> ε-NFA

In [746]:
def isapt(c):
    assert len(c) == 1
    if ord(c) >= ord('a') and ord(c) <= ord('z') or ord(c) >= ord('A') and ord(c) <= ord('Z') or ord(c) >= ord('0') and ord(c) <= ord('9'):
        return True
    return False

In [747]:
def isfast(org, top):
    if org == '.' and top == '+':
        return True
    elif org == '*' and top != '*':
        return True
    return False
         

In [748]:
class node:
    def __init__(self, cn, cv, left=None, right=None):
        # cn : 0:+ 1:. 2:* 3:symbols
        self.cn = cn
        self.cv = cv
        self.left = None
        self.right = None
from copy import deepcopy
from pprint import pprint
import json

In [749]:
RE_ARR = ["(a+b)*abb", "(b+a(aa*b)*b)*" ,"(b+aa+ac+aaa+aac)*", "(1+01)*00(0+1)*" ,"(0+1)*011", "(a+b)*","ba*b","(a+b)c","a(bc)*","son+bug*"]

In [750]:
indindind = 9
RE = RE_ARR[indindind]
RE = "son+bug*" #여기 RE입력!
concatall = ""
str_arr = []

for i in range(len(RE)-1):
    str_arr.append(RE[i])
    if isapt(RE[i]):
        if isapt(RE[i+1]) or RE[i+1] == '(':
            str_arr.append('.')
    if (RE[i] == ')' or RE[i] == '*') and (RE[i+1] == '(' or isapt(RE[i+1])):
        str_arr.append('.')
str_arr.append(RE[-1])
concatall = "".join(str_arr)

print(concatall)

s.o.n+b.u.g*


In [751]:
stack = []
post_arr = []

for c in concatall:
    if isapt(c) or c == "*":
        post_arr.append(c)
    elif c == ")":
        while len(stack) > 0 and stack[-1] != "(":
            post_arr.append(stack.pop())
        stack.pop()
    elif c == "(":
        stack.append(c)
    elif len(stack) == 0 or stack[-1] == "(" or isfast(c, stack[-1]):
        
        stack.append(c)
    else:
        while len(stack) > 0 and stack[-1] != "(" and not isfast(c, stack[-1]):
            post_arr.append(stack.pop())
        stack.append(c)
while len(stack) > 0:
    post_arr.append(stack.pop())

postall = "".join(post_arr)
print(postall)

so.n.bu.g*.+


In [752]:
stack = []
for c in postall:
        if c == "+":
            nd = node(0,c)
            nd.right = stack.pop()
            nd.left = stack.pop()
            stack.append(nd)
        elif c == ".":
            nd = node(1,c)
            nd.right = stack.pop()
            nd.left = stack.pop()
            stack.append(nd)
        elif c == "*":
            nd = node(2,c)
            nd.left = stack.pop() 
            stack.append(nd)
        elif c == "(" or c == ")":
            continue  
        else:
            stack.append(node(3,c))

In [753]:
class eNFA:
    def __init__(self):
        self.next = {}

In [754]:
def scan_plus(nd):
    start = eNFA()
    end = eNFA()

    left_eNFA = node_filter(nd.left)
    right_eNFA = node_filter(nd.right)

    start.next['ε'] = [left_eNFA[0], right_eNFA[0]]
    left_eNFA[1].next['ε'] = [end]
    right_eNFA[1].next['ε'] = [end]

    return start, end

def scan_dot(nd):
    left_nfa  = node_filter(nd.left)
    right_nfa = node_filter(nd.right)

    left_nfa[1].next['ε'] = [right_nfa[0]]
    return left_nfa[0], right_nfa[1]

def scan_star(nd):
    start = eNFA()
    end = eNFA()

    starred_nfa = node_filter(nd.left)

    start.next['ε'] = [starred_nfa[0], end]
    starred_nfa[1].next['ε'] = [starred_nfa[0], end]

    return start, end

def scan_symbol(nd):
    start = eNFA()
    end = eNFA()
    
    start.next[nd.cv] = [end]
    return start, end

In [755]:
def node_filter(nd):
    if nd.cn == 0:
        return scan_plus(nd)
    elif nd.cn == 1:
        return scan_dot(nd)
    elif nd.cn == 2:
        return scan_star(nd)
    elif nd.cn == 3:
        return scan_symbol(nd)
    else:
        print("error")

In [756]:
efas = node_filter(stack[0])

In [757]:
print(efas)
enfa = {}

(<__main__.eNFA object at 0x0000027F5EB98EE0>, <__main__.eNFA object at 0x0000027F5EB98F40>)


In [758]:
def sonbug_numbering(integer):
    assert integer < 1000, "too complicate RE"
    if integer > 99:
        return str(integer)
    elif integer > 9:
        return "0" + str(integer)
    else:
        return "00"+str(integer)

In [759]:
def make_enfa(State, States_history, symbols):
    if State in States_history:
        return
    States_history.append(State)
    for sb in list(State.next):
        if sb not in enfa['TerminalSet']:
            enfa['TerminalSet'].add(sb)
        for nest in State.next[sb]:
            if nest not in symbols:
                symbols[nest] = sorted(symbols.values())[-1]+1
                qs = "q" + sonbug_numbering(symbols[nest])
                enfa['StateSet'].add(qs)
            if not ("(q"+sonbug_numbering(symbols[State])+", "+sb+")" in enfa['DeltaFunctions']):
                enfa['DeltaFunctions']["(q"+sonbug_numbering(symbols[State])+", "+sb+")"] = set()
            enfa['DeltaFunctions']["(q"+sonbug_numbering(symbols[State])+", "+sb+")"].add("q"+sonbug_numbering(symbols[nest]))      
        for nest in State.next[sb]:
            make_enfa(nest, States_history, symbols)

In [760]:
enfa['StateSet'] = set()
enfa['TerminalSet'] = set()
enfa['DeltaFunctions'] = {}
enfa['StartState'] = set()
enfa['FinalStateSet'] = set()
q_0 = "q000"
enfa['StateSet'].add(q_0)
make_enfa(efas[0], [], {efas[0]:0})
    

enfa["StartState"].add("q000")
for st in list(enfa["StateSet"]):
    count = 0
    for key,val in enfa['DeltaFunctions'].items():
        for value in list(val):
            if key[1:5] == st and value != st:
                count += 1
    if count == 0 and st not in enfa["FinalStateSet"]:
        enfa["FinalStateSet"].add(st)


In [761]:
print(enfa['FinalStateSet'])

{'q008'}


In [762]:
with open("./enfa"+str(indindind+1)+".json", 'w', encoding='utf-8') as file:
    json.dump(str(enfa), file, indent="\t", ensure_ascii=False)

In [763]:

pprint(enfa)

{'DeltaFunctions': {'(q000, ε)': {'q001', 'q002'},
                    '(q001, s)': {'q003'},
                    '(q002, b)': {'q009'},
                    '(q003, ε)': {'q004'},
                    '(q004, o)': {'q005'},
                    '(q005, ε)': {'q006'},
                    '(q006, n)': {'q007'},
                    '(q007, ε)': {'q008'},
                    '(q009, ε)': {'q010'},
                    '(q010, u)': {'q011'},
                    '(q011, ε)': {'q012'},
                    '(q012, ε)': {'q014', 'q013'},
                    '(q013, g)': {'q015'},
                    '(q014, ε)': {'q008'},
                    '(q015, ε)': {'q014', 'q013'}},
 'FinalStateSet': {'q008'},
 'StartState': {'q000'},
 'StateSet': {'q000',
              'q001',
              'q002',
              'q003',
              'q004',
              'q005',
              'q006',
              'q007',
              'q008',
              'q009',
              'q010',
              'q011',
             

ε-NFA -> DFA

In [764]:
dfa = {}

In [765]:
def listunion(list1, list2):
    return sorted(list(set(list1) | set(list2)))

def get_enfa_delta(state, input):
    if "("+state+", "+input+")" in enfa['DeltaFunctions'].keys():
        return enfa['DeltaFunctions']["("+state+", "+input+")"]
    return False 

In [766]:
dfa['StateSet'] = set()
dfa['TerminalSet'] = set()
dfa['DeltaFunctions'] = {}
dfa['StartState'] = set()
dfa['FinalStateSet'] = set()




candidate = []


for state in enfa['StateSet']: 
    for sigma in enfa['TerminalSet']:
        if sigma != 'ε':
            continue
        q_temp = []
        
        for key,val in enfa['DeltaFunctions'].items():
            for value in val:
                start = key[1:5]
                inp = key[7]
                end = value
                if state == start and sigma == inp:
                    if end not in q_temp:
                        q_temp.append(end)
        if not len(q_temp):
            continue
        q_states = []
        
        q_states.append(state)
        q_states = sorted(q_states)
        q_temp = sorted(q_temp)
        candidate.append([q_states, sigma, q_temp])    


e_match = {}


for i in enfa['StateSet']:
    e_match[tuple([i])] = [i]


no_change = False
while not no_change:
    no_change = True
    for delta in candidate: #single
        if len(delta[0]) == 1 and delta[1] == 'ε' and len(e_match[tuple(delta[0])]) == 1:
            e_match[tuple(delta[0])] = e_match[tuple(delta[0])]+delta[2] # ('q000',) : ["q001","q002"]
            if delta[0][0] not in e_match[tuple(delta[0])]: #본인 넣는다.
                e_match[tuple(delta[0])].append(delta[0][0])
            no_change = False
        if len(delta[0]) == 1 and delta[1] == 'ε':
            temp = deepcopy(e_match[tuple(delta[0])])
            
            for get in e_match[tuple(delta[0])]:
                
                if (get,) in e_match.keys(): 
                    temp = listunion(e_match[(get,)],temp)     
            if len(temp) > len(e_match[tuple(delta[0])]):
                e_match[tuple(delta[0])] = deepcopy(temp) 
                no_change = False

def multcandidate(states):

    for sigma in enfa['TerminalSet']:
        if sigma != 'ε':
            continue
        q_temp = []
        for state in states:
            for key,val in enfa['DeltaFunctions'].items():
                for value in val:
                    start = key[1:5]
                    inp = key[7]
                    end = value
                    if state == start and sigma == inp:
                        if end not in q_temp:
                            q_temp.append(end)
        if not len(q_temp):
            continue

        
        q_states = states
        q_states = sorted(q_states)
        q_temp = sorted(q_temp)
        candidate.append([q_states, sigma, q_temp])  

    no_change = False
    while not no_change:
        no_change = True
        for delta in candidate: #mult
            
            if len(delta[0]) != 1 and delta[1] == 'ε' and tuple(delta[0]) not in e_match.keys():
                e_match[tuple(delta[0])] = deepcopy(delta[2]) # ('q000','q001') : ["q001","q002"]
                no_change = False
            if len(delta[0]) != 1 and delta[1] == 'ε':
                temp = deepcopy(e_match[tuple(delta[0])])
                for d in delta[0]:
                    for get in e_match[(d,)]:
                        if (get,) in e_match.keys(): 
                            temp=listunion(e_match[(get,)],temp)
                if len(temp) > len(e_match[tuple(delta[0])]):
                    e_match[tuple(delta[0])] = deepcopy(temp) 
                    no_change = False



new_candidate = []
for ss in enfa['StartState']:
    dfa['StartState'].add(tuple(sorted(e_match[(ss,)])))
    dfa['StateSet'].add(tuple(sorted(e_match[(ss,)])))
    no_change = False
    have_to_do = []
    have_to_do.append((ss,))
    while len(have_to_do):
        tg = have_to_do.pop()
        
        for sigma in enfa['TerminalSet']:
            if sigma == 'ε':
                continue
            after_closer_result = []
            
            if tuple(tg) not in e_match.keys():
                
                multcandidate(tg)

            for ecs in e_match[tuple(tg)]:
                
                result = get_enfa_delta(ecs, sigma)
                if result:

                    for r in result:
                        if (r,) in e_match.keys():
                            after_closer_result = listunion(after_closer_result, e_match[(r,)])
            if not len(after_closer_result): # 결과가 없을떄
                continue 
            if tuple(after_closer_result) not in dfa['StateSet']:
                dfa['StateSet'].add(tuple(after_closer_result))
                have_to_do.append(after_closer_result)
            new_candidate.append([sorted(e_match[tuple(tg)]),sigma,sorted(after_closer_result)])



for states in dfa['StateSet']:
    for state in states:
        if state in enfa['FinalStateSet'] and states not in dfa['FinalStateSet']:
            dfa['FinalStateSet'].add(states)

enfa['TerminalSet'].remove('ε')
dfa['TerminalSet'] = deepcopy(enfa['TerminalSet'])



In [767]:
pprint(new_candidate)

[[['q000', 'q001', 'q002'], 's', ['q003', 'q004']],
 [['q000', 'q001', 'q002'], 'b', ['q009', 'q010']],
 [['q009', 'q010'], 'u', ['q008', 'q011', 'q012', 'q013', 'q014']],
 [['q008', 'q011', 'q012', 'q013', 'q014'],
  'g',
  ['q008', 'q013', 'q014', 'q015']],
 [['q008', 'q013', 'q014', 'q015'], 'g', ['q008', 'q013', 'q014', 'q015']],
 [['q003', 'q004'], 'o', ['q005', 'q006']],
 [['q005', 'q006'], 'n', ['q007', 'q008']]]


In [768]:
reduce_table = {}
for i,v in enumerate(dfa['StateSet']):
    val = "q"+sonbug_numbering(i)
    reduce_table[v] = val
print(reduce_table)

{('q008', 'q011', 'q012', 'q013', 'q014'): 'q000', ('q007', 'q008'): 'q001', ('q008', 'q013', 'q014', 'q015'): 'q002', ('q005', 'q006'): 'q003', ('q003', 'q004'): 'q004', ('q009', 'q010'): 'q005', ('q000', 'q001', 'q002'): 'q006'}


In [769]:
for delta in new_candidate:
    dfa['DeltaFunctions']["("+reduce_table[tuple(sorted(delta[0]))]+", "+delta[1]+")"] = {reduce_table[tuple(sorted(delta[2]))]}

sett = set()
sett.add(reduce_table[list(dfa['StartState'])[0]])

dfa['StartState'] = deepcopy(sett)

sett = set()

for state in dfa['StateSet']:
    sett.add(reduce_table[state])
    
dfa['StateSet'] = deepcopy(sett)

sett = set()

for state in dfa['FinalStateSet']:
    sett.add(reduce_table[state])
dfa['FinalStateSet'] = sett


In [770]:

print(e_match)
e_m = deepcopy(e_match)
for i in e_match.keys():
    e_m[str(i)] = e_m.pop(i)
with open("./e_closer.json", 'w', encoding='utf-8') as file:
    json.dump(e_m, file, indent="\t", ensure_ascii=False)

{('q015',): ['q008', 'q013', 'q014', 'q015'], ('q006',): ['q006'], ('q008',): ['q008'], ('q009',): ['q009', 'q010'], ('q007',): ['q007', 'q008'], ('q001',): ['q001'], ('q012',): ['q008', 'q012', 'q013', 'q014'], ('q011',): ['q008', 'q011', 'q012', 'q013', 'q014'], ('q000',): ['q000', 'q001', 'q002'], ('q010',): ['q010'], ('q014',): ['q014', 'q008'], ('q004',): ['q004'], ('q002',): ['q002'], ('q005',): ['q005', 'q006'], ('q003',): ['q003', 'q004'], ('q013',): ['q013'], ('q009', 'q010'): ['q009', 'q010'], ('q008', 'q011', 'q012', 'q013', 'q014'): ['q008', 'q011', 'q012', 'q013', 'q014'], ('q008', 'q013', 'q014', 'q015'): ['q008', 'q013', 'q014', 'q015'], ('q003', 'q004'): ['q003', 'q004'], ('q005', 'q006'): ['q005', 'q006'], ('q007', 'q008'): ['q007', 'q008']}


In [771]:
pprint(dfa['StartState'])
pprint(dfa['FinalStateSet'])

{'q006'}
{'q001', 'q002', 'q000'}


In [772]:
with open("./dfa"+str(indindind+1)+".json", 'w', encoding='utf-8') as file:
    json.dump(str(dfa), file, indent="\t", ensure_ascii=False)

In [773]:
pprint(dfa)

{'DeltaFunctions': {'(q000, g)': {'q002'},
                    '(q002, g)': {'q002'},
                    '(q003, n)': {'q001'},
                    '(q004, o)': {'q003'},
                    '(q005, u)': {'q000'},
                    '(q006, b)': {'q005'},
                    '(q006, s)': {'q004'}},
 'FinalStateSet': {'q001', 'q002', 'q000'},
 'StartState': {'q006'},
 'StateSet': {'q006', 'q001', 'q000', 'q004', 'q002', 'q005', 'q003'},
 'TerminalSet': {'g', 's', 'o', 'b', 'u', 'n'}}


DFA -> reduced DFA

In [774]:
def get_dfa_delta(state, input):
    if "("+state+", "+input+")" in dfa['DeltaFunctions'].keys():
        return list(dfa['DeltaFunctions']["("+state+", "+input+")"])[0]
    return "-1"

def something_in_list(something, list):
    if something in list:
        return True
    return False 

def combine_same_inlist(lst):
    pass

from collections import defaultdict

In [775]:


getarr=sorted(list(dfa['StateSet']))

arr = [list(dfa['StateSet'] - dfa['FinalStateSet']),sorted(list(dfa['FinalStateSet']))]
LEN_STATE = len(dfa['StateSet'])
checkarr = [["I"]*len(dfa['TerminalSet']) for _ in range(LEN_STATE)]

for i in range(LEN_STATE+1):
    lenarr = len(arr)
    for j in range(LEN_STATE):
        for ki,kv in enumerate(dfa['TerminalSet']):
            anstr=get_dfa_delta(getarr[j],kv)
            
            
            if anstr=="-1":
                checkarr[j][ki] = "NNN"
            else:
                for mi,mv in enumerate(arr):
                    if something_in_list(anstr,mv):
                        checkarr[j][ki] = sonbug_numbering(mi)
    temp_arr = []

    
    big_ddic = []
    for ji,jv in enumerate(arr):
        ddic = defaultdict(list)
        for ki,kv in enumerate(jv):
            sumstr = ""
            for m in checkarr[int(kv[1:])]:
                sumstr += m
            ddic[sumstr].append(kv)
        
        for key,val in ddic.items():
            temp_arr.append(val)
        big_ddic.append(deepcopy(ddic))
        


    arr = deepcopy(temp_arr)    
    
    
    if lenarr == len(arr):
        print(arr)
        print(big_ddic)
        break




[['q006'], ['q005'], ['q003'], ['q004'], ['q000', 'q002'], ['q001']]
[defaultdict(<class 'list'>, {'NNN003NNN001NNNNNN': ['q006']}), defaultdict(<class 'list'>, {'NNNNNNNNNNNN004NNN': ['q005']}), defaultdict(<class 'list'>, {'NNNNNNNNNNNNNNN005': ['q003']}), defaultdict(<class 'list'>, {'NNNNNN002NNNNNNNNN': ['q004']}), defaultdict(<class 'list'>, {'004NNNNNNNNNNNNNNN': ['q000', 'q002']}), defaultdict(<class 'list'>, {'NNNNNNNNNNNNNNNNNN': ['q001']})]


In [776]:
def sbstr_slicing(k,ji):
    return k[ji*3 : ji*3+3]

In [777]:
new_candidate = []
for i in big_ddic:
    for k,v in i.items():
        for ji,jv in enumerate(dfa['TerminalSet']):
            #v가 jv를 보고 arr[k[ji]]로 이동한다는 것임
            if sbstr_slicing(k,ji)!="NNN":
                new_candidate.append([v,jv,arr[int(sbstr_slicing(k,ji))]])
print(new_candidate)


[[['q006'], 's', ['q004']], [['q006'], 'b', ['q005']], [['q005'], 'u', ['q000', 'q002']], [['q003'], 'n', ['q001']], [['q004'], 'o', ['q003']], [['q000', 'q002'], 'g', ['q000', 'q002']]]


In [778]:
rdfa = {}
rdfa['StateSet'] = set()
rdfa['TerminalSet'] = deepcopy(dfa['TerminalSet'])
rdfa['DeltaFunctions'] = {}
rdfa['StartState'] = set()
rdfa['FinalStateSet'] = set()

reduce_table = {}
for i,v in enumerate(arr):
    val = "q"+sonbug_numbering(i)
    reduce_table[tuple(v)] = val
print(reduce_table)

{('q006',): 'q000', ('q005',): 'q001', ('q003',): 'q002', ('q004',): 'q003', ('q000', 'q002'): 'q004', ('q001',): 'q005'}


In [779]:
for delta in new_candidate:
    rdfa['DeltaFunctions']["("+reduce_table[tuple(delta[0])]+", "+delta[1]+")"] = {reduce_table[tuple(delta[2])]}

sett = set()

for states in arr:
    sett.add(reduce_table[tuple(states)])

rdfa['StateSet'] = deepcopy(sett)

sett = set()

for ss in dfa['StartState']:
    for k,v in reduce_table.items():
        if ss in k:
            sett.add(v)

rdfa['StartState'] = deepcopy(sett)

sett = set()

for fs in dfa['FinalStateSet']:
    for k,v in reduce_table.items():
        if fs in k:
            sett.add(v)

rdfa['FinalStateSet'] = deepcopy(sett)



In [780]:
with open("./rdfa"+str(indindind+1)+".json", 'w', encoding='utf-8') as file:
    json.dump(str(rdfa), file, indent="\t", ensure_ascii=False)

In [781]:
pprint(rdfa)

{'DeltaFunctions': {'(q000, b)': {'q001'},
                    '(q000, s)': {'q003'},
                    '(q001, u)': {'q004'},
                    '(q002, n)': {'q005'},
                    '(q003, o)': {'q002'},
                    '(q004, g)': {'q004'}},
 'FinalStateSet': {'q005', 'q004'},
 'StartState': {'q000'},
 'StateSet': {'q001', 'q000', 'q004', 'q002', 'q005', 'q003'},
 'TerminalSet': {'s', 'g', 'o', 'b', 'u', 'n'}}
